In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

C:\Users\tsmar\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
ComicDataRaw = pd.read_csv("C:/Users/tsmar/dc-wikia-data.csv")

In [3]:
ComicDataRaw.fillna(value=0, axis=0, inplace=True)

In [4]:
ComicData2 = ComicDataRaw.drop(labels=['urlslug', 'page_id', 'GSM', 'FIRST APPEARANCE'], axis=1) #drop unwanted cols
len(ComicData2)
#ComicData3 = ComicData2.fillna(0, axis=0) #drop rows with NaN values.
ComicAlignment = ComicDataRaw.iloc[:, 4]

In [5]:
len(ComicAlignment)

6896

In [6]:
#we will be prediction the alignment of a superhero based on other data about them. 

In [7]:
ComicData = ComicDataRaw.drop(labels=['ALIGN', 'urlslug', 'page_id', 'GSM', 'FIRST APPEARANCE'], axis=1)
ComicData = ComicData.fillna(value=0, axis=0)

len(ComicData)

6896

In [8]:
ComicData.head

<bound method NDFrame.head of                                  name               ID         EYE  \
0                Batman (Bruce Wayne)  Secret Identity   Blue Eyes   
1               Superman (Clark Kent)  Secret Identity   Blue Eyes   
2          Green Lantern (Hal Jordan)  Secret Identity  Brown Eyes   
3            James Gordon (New Earth)  Public Identity  Brown Eyes   
4         Richard Grayson (New Earth)  Secret Identity   Blue Eyes   
5         Wonder Woman (Diana Prince)  Public Identity   Blue Eyes   
6              Aquaman (Arthur Curry)  Public Identity   Blue Eyes   
7           Timothy Drake (New Earth)  Secret Identity   Blue Eyes   
8      Dinah Laurel Lance (New Earth)  Public Identity   Blue Eyes   
9                 Flash (Barry Allen)  Secret Identity   Blue Eyes   
10                         GenderTest  Secret Identity   Blue Eyes   
11             Alan Scott (New Earth)  Secret Identity   Blue Eyes   
12         Barbara Gordon (New Earth)  Secret Identity   Blu

In [9]:
#now, this dataframe needs to be converted into numerical data so that machine learning can be run on it. 
#lets build some dictionaries!!!!
eye_map = {'Blue Eyes' : 1, 'Green Eyes' : 2, 'Brown Eyes' : 3, 'Black Eyes' : 4, 'Red Eyes' : 5, 'White Eyes' : 6,
           'Hazel Eyes' : 7,  'Yellow Eyes' : 8, 'Purple Eyes' : 9, 'Orange Eyes' : 10, 'Pink Eyes' : 11, 'Silver Eyes' : 12,
           'Gold Eyes' : 13, 'Grey Eyes' : 14, 'Photocellular Eyes' : 15, 'Amber Eyes' : 16, 'Violet Eyes':17, 'Auburn Hair':20, 0:0}

hair_map = {'Blue Hair' : 1, 'Green Hair' : 2, 'Brown Hair' : 3, 'Black Hair' : 4, 'Red Hair' : 5, 'White Hair' : 6,
           'Hazel Hair' : 7,  'Yellow Hair' : 8, 'Purple Hair' : 9, 'Orange Hair' : 10, 'Pink Hair' : 11, 'Silver Hair' : 12, 
            'Blond Hair' : 13, 'Strawberry Blond Hair' : 14, 'Gold Hair' : 15, 'Grey Hair' : 16, 'Reddish Brown Hair':17, 'Violet Hair':18,
            'Platinum Blond Hair':19, 'Auburn Hair':20, 0:0}

In [10]:
id_map = {'Secret Identity' : 0, 'Public Identity' : 1, 'Identity Unknown': 2, 0:0}
gender_map = {'Male Characters' : 0, 'Female Characters' : 1, 'Genderless Characters': 2, 'Transgender Characters': 3, 0:0}
alive_map = {'Living Characters' : 0, 'Deceased Characters' : 1, 0:0}
align_map = {'Good Characters' : 0, 'Neutral Characters' : 1, 'Bad Characters' : 2, 'Reformed Criminals' : 3, 0:4}

In [11]:
#I will predict if a character is aligned to Good or Evil or Reformed based on these following data items:
    #1. Gender,
    #2. Eye Color
    #3. is Alive
    #4. Number of Appearences
    #5. Year
    #6. simple hash of their name.
    #7. hair color. 

In [12]:
#first we will turn list of alignments into list of numbers: 
import math
num_align = [align_map[d] for d in ComicAlignment] #these are our labels. 
num_gender = [gender_map[g[4]] for g in ComicData.values]
num_eye_color = [eye_map[c[2]] for c in ComicData.values]
num_isAlive = [alive_map[a[5]] for a in ComicData.values]
num_hair = [hair_map[h[3]] for h in ComicData.values]
num_id = [id_map[p[1]] for p in ComicData.values]
num_name = [len(word[0]) for word in ComicData.values]

In [13]:
ComicData.values[0]

array(['Batman (Bruce Wayne)', 'Secret Identity', 'Blue Eyes',
       'Black Hair', 'Male Characters', 'Living Characters', 3093.0,
       1939.0], dtype=object)

In [14]:
ComicData['ID'] = num_id
ComicData['EYE'] = num_eye_color
ComicData['ALIVE'] = num_isAlive
ComicData['HAIR'] = num_hair
ComicData['SEX'] = num_gender
ComicData['name'] = num_name

In [15]:
ComicData['APPEARANCES'] = ComicData['APPEARANCES'] / 4000
ComicData['YEAR'] = ComicData['YEAR'] / 2000
ComicData['name'] = ComicData['name'] /  25
ComicData['EYE'] = ComicData['EYE'] / 20
ComicData['HAIR'] = ComicData['HAIR'] / 20
ComicData.head

<bound method NDFrame.head of       name  ID   EYE  HAIR  SEX  ALIVE  APPEARANCES    YEAR
0     0.80   0  0.05  0.20    0      0      0.77325  0.9695
1     0.84   0  0.05  0.20    0      0      0.62400  0.9930
2     1.04   0  0.15  0.15    0      0      0.39125  0.9795
3     0.96   1  0.15  0.30    0      0      0.32900  0.9935
4     1.08   0  0.05  0.20    0      0      0.30925  0.9700
5     1.08   1  0.05  0.20    1      0      0.30775  0.9705
6     0.88   1  0.05  0.65    0      0      0.28025  0.9705
7     1.00   0  0.05  0.20    0      0      0.27375  0.9945
8     1.20   1  0.05  0.65    1      0      0.26875  0.9845
9     0.76   0  0.05  0.65    0      0      0.25700  0.9780
10    0.40   0  0.05  0.65    1      0      0.25700  0.9780
11    0.88   0  0.05  0.65    0      1      0.24225  0.9700
12    1.04   0  0.05  0.25    1      0      0.23775  0.9835
13    1.00   1  0.05  0.15    0      0      0.23775  0.9700
14    0.84   1  0.05  0.20    1      0      0.23350  0.9690
15    1.16

In [16]:
#so we have our labels: 
len(num_align)

6896

In [17]:
#Now the data is ready! we can go to work!
#lets split up the data into training, validation, testing
TrainSplit = 5/8
ValidSplit = 1/8
TestSplit = 2/8

data_size = 6896
train_len = int(data_size * TrainSplit)
valid_len = int(data_size * ValidSplit)
test_len = int(data_size * TestSplit)

TrainSet = ComicData.values[:train_len]
TrainLabels = num_align[:train_len]

ValidationSet = ComicData.values[train_len+1:train_len+valid_len+1]
ValidationLabels = num_align[train_len+1:train_len+valid_len+1]

TestSet = ComicData.values[train_len + valid_len:]
TestLabels = num_align[train_len + valid_len:]

In [18]:
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras.layers import Input, Dense, Activation, Dropout
from keras.models import Sequential
from keras.models import Model

Using TensorFlow backend.


In [19]:
model = Sequential()
model.add(Dense(64, input_dim=8, activation='relu', init='uniform'))
model.add(Dropout(rate=0.125))
model.add(Dense(32, activation='relu', init='uniform'))
model.add(Dropout(rate=0.125))
model.add(Dense(16, activation='relu', init='uniform'))
model.add(Dropout(rate=0.125))
model.add(Dense(8, activation='relu', init='uniform'))
model.add(Dense(5, activation='softmax', init='uniform'))
model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, input_dim=8, activation="relu", kernel_initializer="uniform")`
  
C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(32, activation="relu", kernel_initializer="uniform")`
  after removing the cwd from sys.path.
C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(16, activation="relu", kernel_initializer="uniform")`
  
C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(8, activation="relu", kernel_initializer="uniform")`
  
C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(5, activation="softmax", kernel_initializer="uniform")`
  

In [20]:
len(TrainLabels)

4310

In [21]:
TrainSet = np.reshape(TrainSet, (train_len, 8))
TrainLabelSet = []
for t in TrainLabels:
    if t == 0:
        TrainLabelSet.append([1, 0, 0, 0, 0])
    elif t == 1:
        TrainLabelSet.append([0, 1, 0, 0, 0])
    elif t == 2:
        TrainLabelSet.append([0, 0, 1, 0, 0])
    elif t == 3:
        TrainLabelSet.append([0, 0, 0, 1, 0])
    elif t == 4:
        TrainLabelSet.append([0, 0, 0, 0, 1])
ValidationSet = np.reshape(ValidationSet, (valid_len, 8))
ValidLabelSet = []
for t in ValidationLabels:
    if t == 0:
        ValidLabelSet.append([1, 0, 0, 0, 0])
    elif t == 1:
        ValidLabelSet.append([0, 1, 0, 0, 0])
    elif t == 2:
        ValidLabelSet.append([0, 0, 1, 0, 0])
    elif t == 3:
        ValidLabelSet.append([0, 0, 0, 1, 0])
    elif t == 4:
        ValidLabelSet.append([0, 0, 0, 0, 1])
        
TestSet = np.reshape(TestSet, (test_len, 8))
TestLabelSet = []
for t in TestLabels:
    if t == 0:
        TestLabelSet.append([1, 0, 0, 0, 0])
    elif t == 1:
        TestLabelSet.append([0, 1, 0, 0, 0])
    elif t == 2:
        TestLabelSet.append([0, 0, 1, 0, 0])
    elif t == 3:
        TestLabelSet.append([0, 0, 0, 1, 0])
    elif t == 4:
        TestLabelSet.append([0, 0, 0, 0, 1])

In [22]:
model.fit(TrainSet, np.array(TrainLabelSet), epochs=100, verbose=1)

Epoch 1/100
4310/4310 [==============================] - 3s 725us/step - loss: 1.3481 - acc: 0.4473
Epoch 2/100
4310/4310 [==============================] - 0s 74us/step - loss: 1.1559 - acc: 0.4680
Epoch 3/100
4310/4310 [==============================] - 0s 75us/step - loss: 1.1506 - acc: 0.4680
Epoch 4/100
4310/4310 [==============================] - 0s 71us/step - loss: 1.1511 - acc: 0.4680
Epoch 5/100
4310/4310 [==============================] - 0s 75us/step - loss: 1.1472 - acc: 0.4680
Epoch 6/100
4310/4310 [==============================] - 0s 71us/step - loss: 1.1510 - acc: 0.4543
Epoch 7/100
4310/4310 [==============================] - 0s 70us/step - loss: 1.1494 - acc: 0.4680
Epoch 8/100
4310/4310 [==============================] - 0s 78us/step - loss: 1.1482 - acc: 0.4680
Epoch 9/100
4310/4310 [==============================] - 0s 75us/step - loss: 1.1481 - acc: 0.4680
Epoch 10/100
4310/4310 [==============================] - 0s 73us/step - loss: 1.1488 - acc: 0.4680
Epoch 11

In [58]:
model.fit(ValidationSet, np.array(ValidLabelSet), epochs=30, verbose=1)

Epoch 1/30
862/862 [==============================] - 0s 62us/step - loss: 1.1332 - acc: 0.5104
Epoch 2/30
862/862 [==============================] - 0s 84us/step - loss: 1.1288 - acc: 0.5139
Epoch 3/30
862/862 [==============================] - 0s 84us/step - loss: 1.1171 - acc: 0.5174
Epoch 4/30
862/862 [==============================] - 0s 72us/step - loss: 1.1208 - acc: 0.5220
Epoch 5/30
862/862 [==============================] - 0s 83us/step - loss: 1.1190 - acc: 0.5139
Epoch 6/30
862/862 [==============================] - 0s 86us/step - loss: 1.1252 - acc: 0.5244
Epoch 7/30
862/862 [==============================] - 0s 96us/step - loss: 1.1203 - acc: 0.5360
Epoch 8/30
862/862 [==============================] - 0s 78us/step - loss: 1.1316 - acc: 0.5093
Epoch 9/30
862/862 [==============================] - 0s 76us/step - loss: 1.1217 - acc: 0.5244
Epoch 10/30
862/862 [==============================] - 0s 83us/step - loss: 1.1217 - acc: 0.5278
Epoch 11/30
862/862 [==================

In [23]:
scores = model.evaluate(TestSet, np.array(TestLabelSet))
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

1724/1724 [==============================] - 0s 167us/step

acc: 53.19%


In [43]:
#now lets try a randomforest
import sklearn
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(criterion='entropy', n_estimators=100, random_state=1, n_jobs=2)
forest.fit(TrainSet, TrainLabelSet)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=2,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

In [44]:
yhat = forest.predict(TestSet)

In [45]:
def label_match(yh, ya):
    for i in range(len(yh)):
        if yh[i] - ya[i] != 0.0:
            return False
    return True

In [46]:
res = [label_match(yhat[i], np.array(TestLabelSet[i])) for i in range(len(yhat))]

In [47]:
sum(res) / len(res)

0.3874709976798144